In [22]:
import pandas as pd
import numpy as np
from statsmodels.api import Logit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [23]:
df = pd.read_csv("C:/Users/user/Downloads/datascience/Python/iris.csv")
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [24]:
df["Species"]

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: Species, Length: 150, dtype: object

In [25]:
df["is_setosa"] = (df["Species"] == "setosa") + 0
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [26]:
model = Logit(endog = df["is_setosa"], exog = df.iloc[:,:2]).fit()

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


In [27]:
model.params

Sepal.Length    -7.529945
Sepal.Width     13.130734
dtype: float64

In [28]:
model.pvalues

Sepal.Length    0.000828
Sepal.Width     0.000989
dtype: float64

In [29]:
pred = model.predict(df.iloc[:3,:2])
pred

0    0.999477
1    0.923824
2    0.998678
dtype: float64

In [30]:
(pred > 0.5) + 0

0    1
1    1
2    1
dtype: int32

In [32]:
model = LogisticRegression(random_state = 123)
model.fit(X = df.iloc[:,:2], y = df["is_setosa"])

LogisticRegression(random_state=123)

In [33]:
model.coef_

array([[-3.38829757,  3.1645277 ]])

In [35]:
model.intercept_

array([8.32330389])

In [39]:
pred = model.predict_proba(df.iloc[:3,:2])
pred = pred[:,1]
pred

array([0.89272024, 0.77104635, 0.92586179])

In [40]:
(pred > 0.5) + 0

array([1, 1, 1])

In [43]:
pred = model.predict_proba(df.iloc[:,:2])
pred = pred[:,1]
pred[:10]

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096,
       0.91436651, 0.97058885, 0.89484454, 0.93034007, 0.82210603])

In [44]:
from sklearn.metrics import roc_auc_score

In [46]:
roc_auc_score(y_true = df["is_setosa"], y_score = pred)

0.9999999999999999

In [48]:
accuracy_score(y_true = df["is_setosa"], y_pred = (pred > 0.9) + 0)

0.8333333333333334

#1 독립변수를 혈압, 혈당, BMI, 인슐린으로 하고 종속변수를 당뇨 여부로 할 경우 분류 정확도는 얼마인가?
*diabetes.csv 파일 사용
*statsmodels 함수 사용
*데이터는 학습:평가 = 8:2로 분리 후 계산
*seed는 123

In [51]:
df = pd.read_csv("C:/Users/user/Downloads/datascience/Python/diabetes.csv")
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size = 0.8, random_state = 123)

In [53]:
model = Logit(endog = df_train["Outcome"], exog = df_train.loc[:,["BloodPressure", "Glucose", "BMI", "Insulin"]]).fit()

Optimization terminated successfully.
         Current function value: 0.626579
         Iterations 5


In [55]:
pred = model.predict(df_test.loc[:,["BloodPressure", "Glucose", "BMI", "Insulin"]])

In [56]:
accuracy_score(y_true = df_test["Outcome"], y_pred = (pred > 0.5) + 0)

0.7012987012987013

#2 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 나이의 승산비는 얼마인가?
*diabetes.csv 파일 사용
*statsmodels 함수 사용

In [57]:
model = Logit(endog = df["Outcome"], exog = df.loc[:, ["Glucose", "BMI", "Age"]]).fit()

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


In [66]:
model.params

Glucose    0.009368
BMI       -0.035639
Age       -0.012898
dtype: float64

In [63]:
np.exp(model.params["Age"])

0.9871844697218117

#3 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 모델의 AUC는 얼마인가?
*diabetes.csv 파일 사용
*statsmodels 함수 사용

In [67]:
model = Logit(endog = df["Outcome"], exog = df.loc[:,["Glucose", "BMI", "Age"]]).fit()

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


In [68]:
roc_auc_score(y_true = df["Outcome"], y_score = model.predict(df.loc[:,["Glucose", "BMI", "Age"]]))

0.5414253731343283